# German EV Charging Infrastructure Analysis

Simple e2e data pipeline using DuckDB.

**Pipeline**: Ingestion → Transformation → Visualization → Export

**Data Sources**: 
- EV stations: OpenChargeMap API
- Population: German postal codes CSV
- States: German PLZ-to-Bundesland mapping


In [1]:
# Setup
%pip install duckdb==1.3.2 lonboard

import duckdb
import os
from lonboard import Map, ScatterplotLayer, viz

# Connect to DuckDB
con = duckdb.connect()
con.sql("INSTALL spatial; LOAD spatial;")
con.sql("INSTALL httpfs; LOAD httpfs;")
print("✅ Setup complete")


Note: you may need to restart the kernel to use updated packages.
✅ Setup complete


## 🔄 1. INGESTION


In [2]:
# EV Charging Stations (Germany)
poi_url = (
    "https://api-01.openchargemap.io/v3/poi"
    "?client=ocm-data-export&maxresults=100000&compact=true&verbose=false"
    "&boundingbox=(55.099,5.866),(47.270,15.041)"
)
print(poi_url)
con.sql(f"CREATE OR REPLACE TABLE raw_poi_de AS SELECT * FROM read_json_auto('{poi_url}');")


https://api-01.openchargemap.io/v3/poi?client=ocm-data-export&maxresults=100000&compact=true&verbose=false&boundingbox=(55.099,5.866),(47.270,15.041)


In [3]:

# German Population by Postal Code
plz_csv_url = "https://downloads.suche-postleitzahl.org/v2/public/plz_einwohner.csv"
con.sql(f"""
CREATE OR REPLACE TABLE raw_plz_population AS
SELECT
  CAST(plz AS VARCHAR) AS plz,
  CAST(einwohner AS BIGINT) AS population,
  CAST(lat AS DOUBLE) AS latitude,
  CAST(lon AS DOUBLE) AS longitude,
  note AS location_name
FROM read_csv_auto('{plz_csv_url}');
""")

# German States Mapping
states_csv_url = "https://gist.githubusercontent.com/jbspeakr/4565964/raw/4083f8b8933f0e9a64dafc943ecbae496f9d65d2/German-Zip-Codes.csv"
con.sql(f"""
CREATE OR REPLACE TABLE raw_plz_states AS
SELECT
  TRIM(CAST(Plz AS VARCHAR)) AS plz,
  TRIM(CAST(Bundesland AS VARCHAR)) AS bundesland
FROM read_csv_auto('{states_csv_url}', delim=';', header=true)
WHERE Plz IS NOT NULL AND Bundesland IS NOT NULL;
""")

print(f"📊 Ingested: {con.sql('SELECT COUNT(*) FROM raw_poi_de').fetchone()[0]:,} EV stations")
print(f"📊 Ingested: {con.sql('SELECT COUNT(*) FROM raw_plz_population').fetchone()[0]:,} postal codes")
print(f"📊 Total German population: {con.sql('SELECT SUM(population) FROM raw_plz_population').fetchone()[0]:,}")


📊 Ingested: 27,504 EV stations
📊 Ingested: 8,170 postal codes
📊 Total German population: 80,322,172


## ⚙️ 2. TRANSFORMATION


In [4]:
# Clean and normalize EV stations
con.sql("""
CREATE OR REPLACE TABLE clean_ev_stations AS
WITH flattened AS (
  SELECT
    ID AS site_id,
    AddressInfo.Title AS title,
    SUBSTR(REGEXP_REPLACE(AddressInfo.Postcode, '[^0-9]', ''), 1, 5) AS plz,
    COALESCE(AddressInfo.Town, AddressInfo.StateOrProvince) AS city,
    CAST(AddressInfo.Latitude AS DOUBLE) AS latitude,
    CAST(AddressInfo.Longitude AS DOUBLE) AS longitude,
    unnest(Connections) AS connection
  FROM raw_poi_de
  WHERE AddressInfo.Latitude IS NOT NULL
    AND AddressInfo.Longitude IS NOT NULL
    AND AddressInfo.Postcode IS NOT NULL
)
SELECT
  site_id, title, plz, city, latitude, longitude,
  SUM(COALESCE(connection.Quantity, 1)) AS connector_count,
  ST_Point(longitude, latitude) AS geometry
FROM flattened
GROUP BY site_id, title, plz, city, latitude, longitude;
""")

# Enrich with population and state data
con.sql("""
CREATE OR REPLACE TABLE enriched_ev_stations AS
SELECT
  e.*, p.population, p.location_name, s.bundesland
FROM clean_ev_stations e
LEFT JOIN raw_plz_population p ON e.plz = p.plz
LEFT JOIN raw_plz_states s ON e.plz = s.plz;
""")

print(f"🔌 Cleaned: {con.sql('SELECT COUNT(*) FROM enriched_ev_stations').fetchone()[0]:,} EV stations")
print(f"🔌 Total connectors: {con.sql('SELECT SUM(connector_count) FROM enriched_ev_stations').fetchone()[0]:,}")


🔌 Cleaned: 56,197 EV stations
🔌 Total connectors: 153,548


In [5]:
# Calculate KPIs
con.sql("""
CREATE OR REPLACE TABLE kpi_by_plz AS
SELECT
  e.plz,
  MAX(e.location_name) AS location_name,
  COUNT(e.site_id) AS ev_stations,
  SUM(e.connector_count) AS total_connectors,
  MAX(e.population) AS population,
  MAX(e.bundesland) AS bundesland,
  ROUND(100000.0 * SUM(e.connector_count) / NULLIF(MAX(e.population), 0), 2) AS connectors_per_100k
FROM enriched_ev_stations e
WHERE e.population IS NOT NULL
GROUP BY e.plz
HAVING SUM(e.connector_count) > 0
ORDER BY connectors_per_100k DESC;
""")

con.sql("""
CREATE OR REPLACE TABLE kpi_by_state AS
SELECT
  e.bundesland,
  COUNT(e.site_id) AS ev_stations,
  SUM(e.connector_count) AS total_connectors,
  SUM(e.population) AS total_population,
  ROUND(100000.0 * SUM(e.connector_count) / NULLIF(SUM(e.population), 0), 2) AS connectors_per_100k
FROM enriched_ev_stations e
WHERE e.population IS NOT NULL AND e.bundesland IS NOT NULL
GROUP BY e.bundesland
ORDER BY connectors_per_100k DESC;
""")

print("📊 KPIs calculated for postal codes and states")


📊 KPIs calculated for postal codes and states


## 📊 3. VISUALIZATION


In [6]:
!pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.5 MB/s  0:00:003.6 MB/s eta 0:00:01:01


In [7]:
# Basic EV stations map
basic_query = con.sql("""
SELECT 
  site_id, title, connector_count, geometry
FROM enriched_ev_stations
WHERE latitude BETWEEN 47.0 AND 55.2
  AND longitude BETWEEN 5.5 AND 15.5
ORDER BY connector_count DESC
LIMIT 5000
""")

basic_map = viz(basic_query)
display(basic_map)
print("🗺️ Basic EV stations map created")


/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

🗺️ Basic EV stations map created


## 💾 4. EXPORT


In [8]:
# Create separate database and export as Parquet
con.sql("ATTACH 'ev_germany_analysis.db' AS ev_db;")

# Copy tables to new database
tables_to_export = [
    'enriched_ev_stations',
    'kpi_by_plz', 
    'kpi_by_state',
    'raw_plz_population'
]

for table in tables_to_export:
    con.sql(f"CREATE OR REPLACE TABLE ev_db.{table} AS SELECT * FROM {table};")
    # Export to Parquet
    con.sql(f"COPY ev_db.{table} TO '{table}.parquet' (FORMAT PARQUET);")
    row_count = con.sql(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"✅ Exported {table}.parquet ({row_count:,} rows)")

print("\n💾 All data exported to:")
print("  📁 ev_germany_analysis.db (DuckDB database)")
print("  📄 *.parquet files (individual tables)")


✅ Exported enriched_ev_stations.parquet (56,197 rows)
✅ Exported kpi_by_plz.parquet (5,661 rows)
✅ Exported kpi_by_state.parquet (16 rows)
✅ Exported raw_plz_population.parquet (8,170 rows)

💾 All data exported to:
  📁 ev_germany_analysis.db (DuckDB database)
  📄 *.parquet files (individual tables)


## 📈 5. SUMMARY


In [9]:
# Fixed population statistics
overall_stats = con.sql("""
SELECT 
  COUNT(*) as total_stations,
  SUM(connector_count) as total_connectors,
  COUNT(DISTINCT e.plz) as unique_postal_codes,
  SUM(DISTINCT_population.pop) as actual_population_covered,
  ROUND(AVG(connector_count), 1) as avg_connectors_per_station
FROM enriched_ev_stations e
JOIN (
  SELECT plz, MAX(population) as pop 
  FROM enriched_ev_stations 
  WHERE population IS NOT NULL 
  GROUP BY plz
) DISTINCT_population ON e.plz = DISTINCT_population.plz
WHERE e.population IS NOT NULL
""").fetchone()

print("📊 FINAL SUMMARY")
print("=" * 40)
print(f"🚗 Total EV Stations: {overall_stats[0]:,}")
print(f"🔌 Total Connectors: {overall_stats[1]:,}")
print(f"📮 Postal Codes Covered: {overall_stats[2]:,}")
print(f"👥 Population Covered: {overall_stats[3]:,}")
print(f"⚡ Avg Connectors/Station: {overall_stats[4]}")

# Top areas
print("\n🏆 TOP 5 AREAS (connectors per 100k residents):")
top_areas = con.sql("SELECT plz, location_name, connectors_per_100k FROM kpi_by_plz LIMIT 5").fetchall()
for i, area in enumerate(top_areas, 1):
    print(f"  {i}. PLZ {area[0]} - {area[2]} connectors/100k")

print("\n🏛️ TOP 5 STATES:")
top_states = con.sql("SELECT bundesland, connectors_per_100k FROM kpi_by_state LIMIT 5").fetchall()
for i, state in enumerate(top_states, 1):
    print(f"  {i}. {state[0]} - {state[1]} connectors/100k")

print("\n✅ Pipeline complete! Data ready for analysis.")


📊 FINAL SUMMARY
🚗 Total EV Stations: 53,288
🔌 Total Connectors: 146,583
📮 Postal Codes Covered: 5,661
👥 Population Covered: 890,898,192
⚡ Avg Connectors/Station: 2.8

🏆 TOP 5 AREAS (connectors per 100k residents):
  1. PLZ 60549 - 625000.0 connectors/100k
  2. PLZ 70629 - 362500.0 connectors/100k
  3. PLZ 64743 - 266666.67 connectors/100k
  4. PLZ 30521 - 43750.0 connectors/100k
  5. PLZ 83364 - 30769.23 connectors/100k

🏛️ TOP 5 STATES:
  1. Mecklenburg-Vorpommern - 22.76 connectors/100k
  2. Rheinland-Pfalz - 22.58 connectors/100k
  3. Bayern - 21.86 connectors/100k
  4. Hamburg - 21.3 connectors/100k
  5. Schlewig-Holstein - 17.92 connectors/100k

✅ Pipeline complete! Data ready for analysis.
